In [1]:
# %% [markdown]
# ## Agentic Architecture Implementation
# Autonomous problem-solving system combining Llama 3.2 3B with real-time web access

# %%
# Install required libraries


In [2]:
# %% [markdown]
# ## Autonomous Research Agent with DuckDuckGo Integration
# Free alternative implementation using privacy-focused search

# %%
# Install dependencies
!pip install -q transformers huggingface-hub langgraph duckduckgo-search
!pip install -q transformers huggingface-hub langgraph duckduckgo-search brave-search-python-client


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langflow-base 0.1.3 requires orjson==3.10.0, but you have orjson 3.10.15 which is incompatible.
litellm 1.59.8 requires httpx<0.28.0,>=0.23.0, but you have httpx 0.28.1 which is incompatible.
storage3 0.7.7 requires httpx[http2]<0.28,>=0.24, but you have httpx 0.28.1 which is incompatible.
brave-search 0.2.0 requires httpx<0.26.0,>=0.24.0, but you have httpx 0.28.1 which is incompatible.
brave-search 0.2.0 requires tenacity<9.0.0,>=8.2.3, but you have tenacity 9.0.0 which is incompatible.
supabase 2.6.0 requires httpx<0.28,>=0.24, but you have httpx 0.28.1 which is incompatible.
qianfan 0.3.5 requires tenacity<9.0.0,>=8.2.3, but you have tenacity 9.0.0 which is incompatible.
astra-assistants 2.2.9 requires httpx<0.28.0,>=0.27.0, but you have httpx 0.28.1 which is incompatible.
langchain-google-vertexai 2.0.7 requi

# %% [markdown]
# ### 1. Environment Setup
# Configure API keys and core components

# %%


In [ ]:


# %% [markdown]
# ### 1. System Initialization

# %%
import os
from getpass import getpass
from duckduckgo_search import DDGS
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langgraph.graph import END, StateGraph

# Configure environment
os.environ["HF_TOKEN"] = ""


2025-02-18 21:31:55.261614: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-18 21:31:55.374399: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739935915.420168     755 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739935915.433765     755 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-18 21:31:55.540148: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# %% [markdown]
# ### 2. Search Module Implementation

# %%
class AcademicSearch:
    def __init__(self, max_results: int = 5):
        self.max_results = max_results

    def search(self, query: str) -> list:
        """Execute academic-focused web search"""
        with DDGS() as ddgs:
            results = ddgs.text(
                keywords=query,
                region='us-en',
                max_results=self.max_results
            )

        return self._filter_results(results)

    def _filter_results(self, results: list) -> list:
        """Prioritize academic sources and relevant content"""
        filtered = []
        for res in results:
            if '.edu' in res['href'] or 'academic' in res['title'].lower():
                filtered.append({
                    'title': res['title'],
                    'url': res['href'],
                    'content': res['body']
                })
        return filtered[:3]  # Return top 3 academic results

In [3]:
# %% [markdown]
# ### 3. LLM Configuration

# %%
def load_llama_pipeline():
    """Initialize quantized Llama 3.2 3B model"""
    model_id = "meta-llama/Llama-3.2-3B-Instruct"

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        load_in_4bit=True,
        attn_implementation="flash_attention_2"
    )

    return pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=1024
    )

llama_pipeline = load_llama_pipeline()


print("\n✅ Verification Complete - All sources from .edu domains")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0



✅ Verification Complete - All sources from .edu domains


In [3]:
# %% [markdown] (old based on static keywords)
# ### 4. Agent Orchestration

# %%
from typing import TypedDict, List, Dict

class AgentState(TypedDict):
    query: str
    requires_search: bool
    context: str
    sources: List[Dict]
    response: str

class ResearchAgent:
    def __init__(self):
        self.search_engine = AcademicSearch()
        self.workflow = self._build_workflow()

    def _build_workflow(self):
        """Create LangGraph decision-making workflow"""
        workflow = StateGraph(AgentState)

        # Add nodes
        workflow.add_node("analyze_query", self.analyze_query)
        workflow.add_node("execute_search", self.execute_search)
        workflow.add_node("generate_response", self.generate_response)

        # Define transitions
        workflow.add_edge("execute_search", "generate_response")
        workflow.add_edge("generate_response", END)

        # Conditional routing
        workflow.add_conditional_edges(
            "analyze_query",
            self.route_decision,
            {"search": "execute_search", "direct": "generate_response"}
        )

        workflow.set_entry_point("analyze_query")
        return workflow.compile()

    def analyze_query(self, state: dict) -> dict:
        """Determine if web search is needed"""
        requires_search = any(
            kw in state["query"].lower()
            for kw in ['deadline', 'current', 'update', 'latest']
        )
        return {"requires_search": requires_search}

    def route_decision(self, state: dict) -> str:
        return "search" if state["requires_search"] else "direct"

    def execute_search(self, state: dict) -> dict:
        """Execute search and process results"""
        raw_results = self.search_engine.search(state["query"])
        context = "\n\n".join(
            f"Source {i+1}: {res['content']}"
            for i, res in enumerate(raw_results)
        )
        return {"context": context, "sources": raw_results}

    def generate_response(self, state: dict) -> dict:
        """Generate final answer with citations"""
        prompt = f"""Answer the query using only the provided context.

        Context:
        {state.get('context', 'No additional context')}

        Query: {state["query"]}

        Answer in academic style with source citations:"""

        response = llama_pipeline(
            prompt,
            do_sample=True,
            temperature=0.3
        )[0]["generated_text"]

        return {
            "response": response.split("Answer:")[-1].strip(),
            "sources": state.get("sources", [])
        }

In [4]:
# %% [markdown] (new based on dynamic keywords)
# ### 4. Agent Orchestration

# %%
from typing import TypedDict, List, Dict

class AgentState(TypedDict):
    query: str
    requires_search: bool
    context: str
    sources: List[Dict]
    response: str

class ResearchAgent:
    def __init__(self):
        self.search_engine = AcademicSearch()
        self.workflow = self._build_workflow()

    def _build_workflow(self):
        """Create LangGraph decision-making workflow"""
        workflow = StateGraph(AgentState)

        # Add nodes
        workflow.add_node("analyze_query", self.analyze_query)
        workflow.add_node("execute_search", self.execute_search)
        workflow.add_node("generate_response", self.generate_response)

        # Define transitions
        workflow.add_edge("execute_search", "generate_response")
        workflow.add_edge("generate_response", END)

        # Conditional routing
        workflow.add_conditional_edges(
            "analyze_query",
            self.route_decision,
            {"search": "execute_search", "direct": "generate_response"}
        )

        workflow.set_entry_point("analyze_query")
        return workflow.compile()

    def analyze_query(self, state: dict) -> dict:
        """Determine if web search is needed"""
        requires_search = any(
            kw in state["query"].lower()
            for kw in ['deadline', 'current', 'update', 'latest']
        )
        return {"requires_search": requires_search}

    def route_decision(self, state: dict) -> str:
        return "search" if state["requires_search"] else "direct"

    def execute_search(self, state: dict) -> dict:
        """Execute search and process results"""
        raw_results = self.search_engine.search(state["query"])
        context = "\n\n".join(
            f"Source {i+1}: {res['content']}"
            for i, res in enumerate(raw_results)
        )
        return {"context": context, "sources": raw_results}

    def generate_response(self, state: dict) -> dict:
        """Generate final answer with citations"""
        prompt = f"""Answer the query using only the provided context.

        Context:
        {state.get('context', 'No additional context')}

        Query: {state["query"]}

        Answer in academic style with source citations:"""

        response = llama_pipeline(
            prompt,
            do_sample=True,
            temperature=0.3
        )[0]["generated_text"]

        return {
            "response": response.split("Answer:")[-1].strip(),
            "sources": state.get("sources", [])
        }

In [6]:

# %% [markdown]
# ### 5. Execution Example

# %%
# Initialize agent
research_bot = ResearchAgent()

# Sample academic query
response = research_bot.workflow.invoke({
    "query": "What is last date for graduate scholarship application in JSOM at university of texas at dallas?"
})

# Display formatted results
print("📚 Academic Response:")
print(response["response"])

print("\n🔍 Source Citations:")
for source in response["sources"]:
    print(f"- {source['title']}")
    print(f"  URL: {source['url']}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


📚 Academic Response:
Answer the query using only the provided context.

        Context:
        No additional context

        Query: What is last date for graduate scholarship application in JSOM at university of texas at dallas?

        Answer in academic style with source citations: 

The deadline for the graduate scholarship application in the JSOM (Jewelers of America Foundation) program at the University of Texas at Dallas (UTD) is typically announced in the spring of each year. However, I must emphasize that the specific deadline may vary from year to year, and it is essential to verify the information with the UTD admissions office or the JSOM program administrators for the most accurate and up-to-date information.

According to the UTD website (UTD.edu), the JSOM program is a part of the College of Arts and Sciences, and the application process typically begins in the fall semester (September to November). The application deadline is usually in the spring semester (February 

In [7]:
# %% [markdown]
# ## Transparent Web Content Analysis System
# Adds content inspection and full-page processing

# %%
!pip install -q trafilatura requests beautifulsoup4

# %% [markdown]
# ### 1. Enhanced Content Fetcher

# %%
import requests
from trafilatura import fetch_url, extract
from bs4 import BeautifulSoup
import re

class WebContentAnalyzer:
    def __init__(self, max_pages=3):
        self.max_pages = max_pages
        self.headers = {
            'User-Agent': 'AcademicResearchBot/1.0 (+https://example.edu/bot)'
        }

    def get_full_content(self, url: str) -> dict:
        """Fetch and analyze full webpage content"""
        try:
            response = requests.get(url, headers=self.headers, timeout=10)
            if response.status_code == 200:
                # Extract main content using multiple methods
                soup = BeautifulSoup(response.text, 'html.parser')
                readable = soup.find_all(['article', 'main', 'div.content'])
                
                # Fallback to trafilatura if standard parsing fails
                if not readable:
                    downloaded = fetch_url(url)
                    content = extract(downloaded, include_links=False)
                else:
                    content = ' '.join([elem.get_text() for elem in readable])
                
                return {
                    'success': True,
                    'content': self._clean_text(content),
                    'length': len(content)
                }
        except Exception as e:
            return {'error': str(e)}
        return {'error': 'Failed to fetch content'}

    def _clean_text(self, text: str) -> str:
        """Clean and normalize text content"""
        text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
        return re.sub(r'\[\d+\]', '', text)  # Remove citation markers

# %% [markdown]
# ### 2. Verified Search Pipeline

# %%
from duckduckgo_search import DDGS

class AcademicSearchWithVerification(WebContentAnalyzer):
    def search_with_analysis(self, query: str) -> list:
        """Search with full content verification"""
        with DDGS() as ddgs:
            results = ddgs.text(query, max_results=self.max_pages)
        
        analyzed = []
        for result in results:
            if '.edu' in result['href']:
                content = self.get_full_content(result['href'])
                analyzed.append({
                    'title': result['title'],
                    'url': result['href'],
                    'snippet': result['body'],
                    'full_content': content.get('content', '')[:15000],  # First 15k chars
                    'error': content.get('error'),
                    'length': content.get('length', 0)
                })
        return analyzed

# %% [markdown]
# ### 3. Model Integration with Content Verification

# %%
from transformers import AutoModelForCausalLM, AutoTokenizer

class VerifiedResearchAgent(AcademicSearchWithVerification):
    def __init__(self):
        super().__init__()
        self.model, self.tokenizer = self._load_model()
        
    def _load_model(self):
        model_id = "meta-llama/Llama-3.2-3B-Instruct"
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            load_in_4bit=True
        )
        return model, tokenizer
    
    def generate_with_validation(self, query: str) -> dict:
        """Full process with content visibility"""
        # Step 1: Perform search and content analysis
        search_results = self.search_with_analysis(query)
        
        # Step 2: Prepare context window
        context = []
        for result in search_results:
            context.append(f"SOURCE: {result['url']}")
            context.append(f"CONTENT: {result['full_content'][:3000]} [...]")  # First 3k chars
        
        # Step 3: Generate response
        prompt = f"""Analyze these academic sources and answer the question.
        
        {' '.join(context)}
        
        Question: {query}
        Answer in academic style with citations:"""
        
        inputs = self.tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = self.model.generate(**inputs, max_new_tokens=512)
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        return {
            'response': response.split("Answer:")[-1].strip(),
            'sources': search_results
        }

# %% [markdown]
# ### 4. Usage with Content Inspection

# %%
# Initialize agent
agent = VerifiedResearchAgent()

# Example query
#results = agent.generate_with_validation(
#    "Fall 2025 graduate application deadlines for Computer Science at UT Dallas"
#)

results = agent.generate_with_validation(
    "Last date for OPT application for graduate students at UTD"
)

# Display verification info
print("🔍 Content Analysis Report:")
for idx, source in enumerate(results['sources'], 1):
    print(f"\nSource {idx}:")
    print(f"URL: {source['url']}")
    print(f"Content Length: {source['length']} chars")
    print(f"First 500 characters:\n{source['full_content'][:500]}...")
    if source['error']:
        print(f"⚠️ Error: {source['error']}")

print("\n📝 Model Response:")
print(results['response'])



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


🔍 Content Analysis Report:

Source 1:
URL: https://isso.utdallas.edu/employment-and-internships/is-employment/post-completion-opt/
Content Length: 34497 chars
First 500 characters:
 International Center > International Students and Scholars Office > Employment and Internships > F-1 Employment > Post-Completion OPT Post-Completion OPT Optional Practical Training (OPT) is a possible benefit for F-1 students, available after completing one year of full-time enrollment at a U.S. college or university in the academic year immediately preceding OPT. You must receive OPT I-20 BEFORE filing I-765 with USCISIf you file the I-765 with USCIS before your OPT I-20 is issued, your appli...

Source 2:
URL: https://isso.utdallas.edu/immigration-status/immigration-calendar/
Content Length: 15291 chars
First 500 characters:
 Immigration Calendar International Center > International Students and Scholars Office > Immigration Status > Immigration Calendar Important Note: Failure to meet immigration deadli

In [8]:
results = agent.generate_with_validation(
    "when is the graduation date for spring 25 at university of texas at dallas"
)

# Display verification info
print("🔍 Content Analysis Report:")
for idx, source in enumerate(results['sources'], 1):
    print(f"\nSource {idx}:")
    print(f"URL: {source['url']}")
    print(f"Content Length: {source['length']} chars")
    print(f"First 500 characters:\n{source['full_content'][:500]}...")
    if source['error']:
        print(f"⚠️ Error: {source['error']}")

print("\n📝 Model Response:")
print(results['response'])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


🔍 Content Analysis Report:

Source 1:
URL: https://www.utdallas.edu/academics/calendar/
Content Length: 3695 chars
First 500 characters:
 The University of Texas at Dallas > Academics > Academic Calendar Academic Calendar See important dates in the UT Dallas academic calendar. Related Pages Related Pages Academic Resources Certificates Academic Calendar Degrees Schools Spring 2025 View the Full Calendar Full-Term Session Key EventsDateLast day for regular registrationJan. 16University closed: Martin Luther King Jr. DayJan. 20Classes begin Jan. 21End of late registration and last day to add/swapJan. 28Census Day; Last day to drop ...

Source 2:
URL: https://registrar.utdallas.edu/graduation/commencement-schedule/
Content Length: 461 chars
First 500 characters:
The Spring 2025 School Commencement Ceremonies will take place the week of May 19-22, with the final schedule of ceremonies released by the end of February. January 29, 2025 Priority Graduation Application Deadline February 24, 20